# Analysis of London spendings with python

In this post I want to discuss how you can use python to fetch data from the internet, 
put them in a readable format and gain some interesting insights.

This exercise is motivated by ["Using SQL for Lightweight Data Analysis"](http://schoolofdata.org/2013/03/26/using-sql-for-lightweight-data-analysis/) by Rufus Pollock. Here, I extend Rufus' analysis to a larger dataset and I use different analysis tools.

## The data

The data come from the ["London GLA spending"](https://www.london.gov.uk/about-us/greater-london-authority-gla/spending-money-wisely/our-spending) website, where GLA stands for Greater London Authority. Every month GLA publishes their spendings on Housing Services, Developing, Communities & Intelligence, etc. While writing, the GLA webpage contains 38 csv files with inhomogeneous formatting. There are empty columns and irregularly spaced data. To complicate things, the GLA website keeps changing root address and html design. So, I do not guarantee that the code described below will work in 2 years from now.

The webpage looks like this:

<img src="https://raw.githubusercontent.com/vincepota/GLA/master/notebook/web_screenshot.png" width="600">

where we are interested in the content of the `CSV file` column. 

The strategy is straightforward: 
- scrap the html code of the GLA webpage;
- extract the links to the `.csv` files;
- download all the data and append the results to a `pandas` dataframe;
- Clean the data
- Have some fun with the data

## The code

Let's import some libraries, where the most important is `BeautifulSoup` which allows to handle the html code hiding behind web pages. If you do not have `BeautifulSoup` installed, you can get it via `pip install BeautifulSoup`.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib2
import matplotlib.pylab as plt
import re
import numpy as np
from __future__ import print_function

Let's extract the `html` code from the GLA webpage:

In [2]:
wpage= 'https://www.london.gov.uk/about-us/greater-london-authority-gla/spending-money-wisely/our-spending'

req = urllib2.Request(wpage)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page, 'html5lib')

The `csv` files that we need are contained in `<td>` tags, which are nested inside `<table>` tags.
Some `<td>` tags contain the direct link to the `csv` file, while other `<td>` tags contain a *link* to another webpage which contains the `csv` file. It is rather confusing, but it can be implemented very easily with python:

In [3]:
table = soup.find_all('table')     # Find all <table> tags
thelist = []                       

for t in table:
    if len(t.find_all('th')) > 0:  # Only select tables with csv files
        for a in t.find_all('a', href=True):   # Find all hyperlinks in the table
            thelink = 'https:' + a['href']     
            if len(thelink) < 40:        # If True, thelink is a link to another webpage
                                         # containing the csv file
                    
               req = urllib2.Request(thelink)    # Scrap thelink wepage
               page = urllib2.urlopen(req)
               soup = BeautifulSoup(page, 'html5lib')

               aa = soup.find_all(href = re.compile('.csv'))[0] # Extract the csv file
               thelink = aa['href']
               thelist.append(thelink)   
            else:                        # If the link is a link to the csv file, append the
               thelist.append(thelink)   # results straight away

`thelist` is a list which contains all the direct links to the `csv` files. Note that we have not downloaded the data yet.

In [4]:
print('the list contains', len(thelist), 'csv files')
thelist[0:5]

the list contains 38 csv files


[u'https://www.london.gov.uk/sites/default/files/mayors_250_report_-_2015-16_-_p12_-_combined.csv',
 u'https://www.london.gov.uk/sites/default/files/mayors_250_report_-_2015-16_-_p11_-_combined_fn.csv',
 u'https://www.london.gov.uk/sites/default/files/mayors_250_report_-_2015-16_-_p10_-_combined.csv',
 u'https://www.london.gov.uk/sites/default/files/mayors_250_report_-_2015-16_-_p09_-_combined.csv',
 u'https://www.london.gov.uk/sites/default/files/copy_of_mayors_250_report_-_2015-16_-_p08_-.csv']

We can now download the data. Instead of downloading every `csv` files to disk, one can use `pandas` ability to read `csv` files straight from the internet. 
Before we do that, let's see how the head of a `csv` file looks like in excel:
![excel](https://raw.githubusercontent.com/vincepota/GLA/master/notebook/excel.png)

We want to discard the information stored in the first few lines. The actual data start from where columns names are: `Vendor ID`, `Vendor Name`, `Cost Element`, etc..

The following lines of code loop through the `csv` files stored in `thelist`, discard the data in the file head, and append the results to the the dataframe `df`:

In [5]:
df = pd.DataFrame()
print('Running')
for i, thefile in enumerate(thelist):
     print(i, end="  "),
     tmp = pd.read_csv(thefile, header=None) 

     # Drop rows with all missing values
     tmp.dropna(inplace=True, how='all',axis=1, thresh=10)
    
     # Find the row with the column names     
     ix = np.where(tmp.values == 'Vendor ID')[0][0]
     column_names = tmp.loc[ix]
     
     # Remove summary from file header
     tmp = tmp[(ix + 1) :]
     tmp.columns = column_names
     
     # Drop columns with all missing values
     tmp.dropna(inplace=True, how='all', axis=0)
     tmp.dropna(inplace=True, axis = 0)
     
     # Append results to dataframe
     df = df.append(tmp, ignore_index = True)
print('Done')

Running
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  Done


Here is how the data look like:

In [10]:
print('The datafarme has ', len(df), 'rows')
df.head(3)

The datafarme has  29138 rows


,Amount,Clearing Date,Cost Element,Directorate,Document No,Expenditure Account Code Description,Service Expenditure Analysis,Vendor ID,Vendor Name
0,66253087.0,2016-02-24,544071,RESOURCES,CHAPS649,FUNCTIONAL BODY GRANT PAYMENT,Highways and transport services,10016524,TRANSPORT FOR LONDON
1,20945312.0,2016-02-15,544093,RESOURCES,CHAPS627,NLE - GRANT PMT TO TFL,Highways and transport services,10016524,TRANSPORT FOR LONDON
2,17926156.0,2016-02-22,544073,RESOURCES,CHAPS643,BUSINESS RATE RETENTION-CLG,Highways and transport services,NC,DCLG


Some cleaning and feature engeneering is still required:

In [7]:
df = df[df.columns[2:]] # Remove the first two empty columns

# Amount
def clean_par(text):
    if '(' in text:
        output = ('-' + re.sub('[()]','',text))
    else:
        output = text
    return output

df['Amount'] = df['Amount'].map(lambda x: clean_par(x)) # transform '(123)' -> -123 
df['Amount'] = df['Amount'].map(lambda x: x.replace(',','')).astype(float) # Remove ',' 

# Clearing Date
df['Clearing Date'] = df['Clearing Date'].map(lambda x: pd.Timestamp(x)) # Transform to time series

# Expenditure Account Code Description
df['Expenditure Account Code Description'] = df['Expenditure Account Code Description'].map(lambda x: x.upper()) # Make upper case

#Directorate
mask = ~df['Directorate'].isnull() # consider only non-null values
df.loc[mask,'Directorate'] = df.loc[mask,'Directorate'].map(lambda x: x.upper()) # Make upper case
df.loc[mask,'Directorate'] = df.loc[mask,'Directorate'].map(lambda x: x.replace('&','AND'))
df.loc[mask,'Directorate'] = df.loc[mask,'Directorate'].map(lambda x: x.rstrip()) # Strip white spaces

In [8]:
df.head()

,Amount,Clearing Date,Cost Element,Directorate,Document No,Expenditure Account Code Description,Service Expenditure Analysis,Vendor ID,Vendor Name
0,66253087.0,2016-02-24,544071,RESOURCES,CHAPS649,FUNCTIONAL BODY GRANT PAYMENT,Highways and transport services,10016524,TRANSPORT FOR LONDON
1,20945312.0,2016-02-15,544093,RESOURCES,CHAPS627,NLE - GRANT PMT TO TFL,Highways and transport services,10016524,TRANSPORT FOR LONDON
2,17926156.0,2016-02-22,544073,RESOURCES,CHAPS643,BUSINESS RATE RETENTION-CLG,Highways and transport services,NC,DCLG
3,7155497.0,2016-02-24,544075,HOUSING,5108675974,GRANTS TO EXTERNAL ORGANISATIONS,Capital,15500400,LONDON BOROUGH OF SOUTHWARK
4,3357407.0,2016-02-24,544075,HOUSING,5108678522,GRANTS TO EXTERNAL ORGANISATIONS,Capital,15500235,LONDON BOROUGH OF TOWER HAMLETS


## Analysis